In [1]:
import torch
import os
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50
from torch.utils.data import DataLoader, random_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm


# Define the path to the dataset
data_dir = 'caltech-101\caltech-101\ObjectCategories\ObjectCategories'  # Replace with your path

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
dataset = ImageFolder(root=data_dir, transform=transform)

# Check the class-to-label mapping
print("Class-to-Index Mapping:", dataset.class_to_idx)

# Define the split sizes
train_size = int(0.67 * len(dataset))
test_size = len(dataset) - train_size

# Split the dataset
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Load the ResNet-50 model and remove the last fully connected layer
model = resnet50(weights='IMAGENET1K_V1')
model = torch.nn.Sequential(*(list(model.children())[:-1]))  # Remove the final layer for embeddings
model.eval()  # Set model to evaluation mode

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Function to get embeddings
def get_embeddings(dataset, model):
    embeddings = []
    labels = []
    for img, label in tqdm(dataset):
        img = img.unsqueeze(0).to(device)
        with torch.no_grad():
            embedding = model(img).squeeze().cpu().numpy()
        embeddings.append(embedding)
        labels.append(label)
    return np.array(embeddings), np.array(labels)

# Get embeddings for both train and test sets
print("Generating train embeddings...")
train_embeddings, train_labels = get_embeddings(train_dataset, model)
print("Generating test embeddings...")
test_embeddings, test_labels = get_embeddings(test_dataset, model)

# Train a Logistic Regression classifier on the embeddings
print("Training Logistic Regression classifier...")
clf = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
clf.fit(train_embeddings, train_labels)

# Evaluate on the test set
print("Evaluating on the test set...")
test_predictions = clf.predict(test_embeddings)

# Calculate accuracy
accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Set Accuracy: {accuracy * 100:.2f}%")


Class-to-Index Mapping: {'BACKGROUND_Google': 0, 'Faces': 1, 'Faces_easy': 2, 'Leopards': 3, 'Motorbikes': 4, 'accordion': 5, 'airplanes': 6, 'anchor': 7, 'ant': 8, 'barrel': 9, 'bass': 10, 'beaver': 11, 'binocular': 12, 'bonsai': 13, 'brain': 14, 'brontosaurus': 15, 'buddha': 16, 'butterfly': 17, 'camera': 18, 'cannon': 19, 'car_side': 20, 'ceiling_fan': 21, 'cellphone': 22, 'chair': 23, 'chandelier': 24, 'cougar_body': 25, 'cougar_face': 26, 'crab': 27, 'crayfish': 28, 'crocodile': 29, 'crocodile_head': 30, 'cup': 31, 'dalmatian': 32, 'dollar_bill': 33, 'dolphin': 34, 'dragonfly': 35, 'electric_guitar': 36, 'elephant': 37, 'emu': 38, 'euphonium': 39, 'ewer': 40, 'ferry': 41, 'flamingo': 42, 'flamingo_head': 43, 'garfield': 44, 'gerenuk': 45, 'gramophone': 46, 'grand_piano': 47, 'hawksbill': 48, 'headphone': 49, 'hedgehog': 50, 'helicopter': 51, 'ibis': 52, 'inline_skate': 53, 'joshua_tree': 54, 'kangaroo': 55, 'ketch': 56, 'lamp': 57, 'laptop': 58, 'llama': 59, 'lobster': 60, 'lotus'

c:\Users\ASUS\miniconda3\envs\neural_networks\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ASUS\miniconda3\envs\neural_networks\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Generating train embeddings...


100%|██████████| 6126/6126 [00:43<00:00, 140.85it/s]


Generating test embeddings...


100%|██████████| 3018/3018 [00:20<00:00, 144.65it/s]
c:\Users\ASUS\miniconda3\envs\neural_networks\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Training Logistic Regression classifier...
Evaluating on the test set...
Test Set Accuracy: 94.00%
